# Authentication

In [ ]:
import json
import requests
import urllib3

urllib3.disable_warnings(category=urllib3.exceptions.InsecureRequestWarning)

# Use this if you are testing against an instance with OPENID auth
base_path= 'http://127.0.0.1:5002/api/v1'
username = "user1"
password = "user1"
%run fetch_openid_bearer.ipynb

# Setup requirements

For a simple test case, we need
* 2 or more agents that can do something
* The id of one of the configured llms

In [ ]:
# Selecting an existing llm module
r = requests.get(f"{base_path}/modules/?type=LLM", verify=False, headers=auth_headers)
print("The following llm modules could be found in your system.")
if len(r.json()) == 0:
  print("You have no llm modules in your system. Please configure a llm module first and then continue by re-executing this cell.")
  exit(1)

for m_json in r.json():
    print(f"\t{m_json.get('name')} - ID: {m_json.get('id')}")

selected_llm_id = r.json()[0].get('id')

print(f"The llm with id {selected_llm_id} has been pre-selected for you. If you wish to take a different llm from the list above, please uncomment the following line and set the id accordingly")

In [ ]:
selected_llm_id = "51a9cf0c-f80d-4d1e-bff2-98b2bab9be3a"

In [ ]:
# Creating Agent 1 - An agent that can only sum up numbers
r = requests.post(f"{base_path}/agent/", verify=False, headers=auth_headers, json={
  "name": "calculator-sum",
  "description": "An agent that knows how to sum up numbers.",
  "systemInstructionIds": [],
  "customSystemInstruction": "The user is giving you multiple numbers in a request, including mathematical numbers like pi or e. Validate that the user wants to sum up the numbers or if his intention is not specified. Then extract all numbers. Then sum up all extracted numbers. Return the sum of all extracted numbers.",
  "llmId": selected_llm_id
})
if r.status_code == 200:
  agent_1_id = r.json().get('id')
  print(f"Agent '{r.json().get('name')}' has been successfully created with ID '{agent_1_id}'.'")
else:
  print(r.status_code)
  print(r.text)
  exit(1)


In [ ]:
# Creating Agent 2 - An agent that can only subtract numbers
r = requests.post(f"{base_path}/agent/", verify=False, headers=auth_headers, json={
  "name": "calculator-subtract",
  "description": "An agent that knows how to subtract numbers.",
  "systemInstructionIds": [],
  "customSystemInstruction": "The user is giving you multiple numbers in a request, including mathematical numbers like pi or e. Validate that the user wants to subtract the numbers or if his intention is not specified. Then extract all numbers. Then subtract each number from each other. Return the subtraction of all extracted numbers.",
  "llmId": selected_llm_id
})
if r.status_code == 200:
  agent_2_id = r.json().get('id')
  print(f"Agent '{r.json().get('name')}' has been successfully created with ID '{agent_2_id}'.'")
else:
  print(r.status_code)
  print(r.text)
  exit(1)

In [ ]:
# Creating Agent 3 - orchestrator
r = requests.post(f"{base_path}/agent/", verify=False, headers=auth_headers, json={
  "name": "maths-orchestrator-agent",
  "description": "An orchestration agent that calls other agents if needed",
  "systemInstructionIds": [],
  "customSystemInstruction": "You receive a mathematical problem from the user. You are equipped with multiple agents to solve different mathematical problems like summarization or subtraction. Use these agents to solve the user problem and return with the final result. If you cannot identify any mathematical problem in the user prompt, use general knowledge to respond.",
  "llmId": selected_llm_id
})
if r.status_code == 200:
  agent_orchestrator_id = r.json().get('id')
  print(f"Agent '{r.json().get('name')}' has been successfully created with ID '{agent_orchestrator_id}'.'")
else:
  print(r.status_code)
  print(r.text)
  exit(1)

Now we need to create the workflow that includes both agents and defines who of these is the root that receives the user prompt first.

In [ ]:
# Creating the workflow
r = requests.post(f"{base_path}/agent-workflow/", verify=False, headers=auth_headers, json={
  "name": "Sum and Subtract Agent Workflow",
  "description": "A workflow to showcase how different agents work together to solve a simple mathematical problem.",
  "agents": [
    {
        'id': agent_1_id,
        'type': 'Agent',
        'canHandOffTo': [
          {
              'id': agent_orchestrator_id,
              'type': 'Agent'
          }
        ]
    },
    {
        'id': agent_2_id,
        'type': 'Agent',
        'canHandOffTo': [
          {
              'id': agent_orchestrator_id,
              'type': 'Agent'
          }
        ]
    },
    {
        'id': agent_orchestrator_id,
        'type': 'Agent',
        'canHandOffTo': None
    }
  ],
  "rootAgent": {
      'id': agent_orchestrator_id,
      'type': 'Agent'
  },
})
if r.status_code == 200:
  agent_workflow_id = r.json().get('id')
  print(f"Agent workflow '{r.json().get('name')}' has been successfully created with ID '{agent_workflow_id}'.'")
else:
  print(r.status_code)
  print(r.text)
  exit(1)

# Chat with the Workflow

First, we need a conversation that is linked to the newly created workflow.

In [ ]:
# Create conversation
r = requests.post(f'{base_path}/agent-workflow/{agent_workflow_id}/conversation', headers=auth_headers, json={
    "llmIds": [selected_llm_id]
})
if r.status_code == 200:
  conversation_id = r.json().get('id')
  print(f"A new conversation has been successfully created with ID '{conversation_id}'.'")
else:
  print(r.status_code)
  print(r.text)
  exit(1)

Now chat with your workflow

In [ ]:
def stream_response(r):
    r.encoding = 'utf-8'
    if r.status_code == 200:
        for line in r.iter_lines():
            print(json.dumps(json.loads(line.decode('utf8')), indent=4))
        print()
    else:
        print(r.text)

In [ ]:
r = requests.post(f'{base_path}/conversation/{conversation_id}/message', headers=auth_headers, json=
{
  "parts": [
    {
      "type": "TEXT",
      "content": "Subtract the following numbers: 308934, 13123, 12.323, pi, 2833. After that, add 10 to the former result."
    }
  ],
  "overrides": []
})
stream_response(r)